# 编曲机器人

## 问题描述 

在学习生活中我们都会用到曲子，但大家用的曲子不免会重合，造成尴尬的场面。而且有时还会遇到版权问题——曲子需要付费才能使用。在网上茫茫曲海中找到一首适合自己的曲子无疑是大海捞针。那怎样才能找到一首免费的风格相近的，且独一无二的曲子呢？为此我研发了“编曲机器人”，来解决这一系列问题。

## 方案描述

1. 针对原曲进行数字化处理，得到曲子的数字队列
1. 数据预处理。准备X输入变量和y变量
1. 把X变量和y变量交给模型进行训练
1. 准备新曲子的初始曲调，对其进行数字化处理。得到数字队列
1. 把数字队列输入训练好的模型，得到模型的输出，作为下一个音符
1. 重复以上步骤，得到所有音符的预测结果形成的数字队列即为机器人创造的新曲子

## 实现过程

### 第一步：导入相关程序包，准备环境
* 本程序使用mingus包+fluidsynth，进行曲谱弹奏
* fluidsynth使用的soundfont: U20PIANO.SF2
* 以下程序需在windows环境下才可运行

### 安装指导

To Be updated


### Import Package

In [1]:
# Windows version
import os
basepath = os.getcwd()
dllspath = os.path.join(basepath, 'fluidsynth', 'bin')
os.environ['PATH'] = dllspath + ';' + os.environ['PATH']


In [2]:
import mingus.core.notes as notes
import mingus.core.progressions as progressions
from mingus.core import chords
from mingus.containers import NoteContainer, Note
from mingus.midi import fluidsynth

In [ ]:
import pandas as pd

### Load the Soundfont

In [3]:
fluidsynth.init("U20PIANO.SF2")

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


True

### 第二步：静态变量及参数准备

静态变量：
* map_int_to_note ：把数字转换成音符
* map_note_to_int ：把音符转换成数字

In [4]:
map_int_to_note = pd.read_csv('int_note.csv', header=0, index_col=0).to_dict()['note']

map_note_to_int = pd.read_csv('int_note.csv', header=0, index_col=1).to_dict()['int']


D-4
1


In [ ]:
# print(map_int_to_note[1])
# print(map_note_to_int['D-4'])

In [5]:
import time

rhythm = 2/4

music = [('A-3',1), ('E-4',1), ('E-4',1), ('E-4',1), ('E-4',0.5), ('D-4',0.5), ('C-4',1), ('D-4',2),
        ('D-4',1.5), ('E-4',0.5), ('D-4',1), ('C-4',1), ('D-4',0.5), ('C-4',0.5), ('A-3',0.5), ('G-3',0.5), ('A-3',2),
        ('E-3',1), ('E-3', 0.5), ('G-3',.5), ('G-3',1), ('G-3',1), ('A-3',1), ('A-3',.5), ('D-4',.5), ('C-4', 2),
        ('A-3',1), ('A-3', 0.5), ('C-4',.5), ('D-4',.5), ('E-4',.5), ('C-4',.5), ('D-4',.5), ('E-4',2), ('E-4',2)]

for i, r in music:
    n = Note(i)
    n.channel = 1
    n.velocity = 100
    fluidsynth.play_Note(n)
    time.sleep(r*rhythm)

In [6]:
int_music = [ (map_note_to_int[x], r) for idx, (x, r) in enumerate(music) ]
print(int_music)

[(-2, 1), (2, 1), (2, 1), (2, 1), (2, 0.5), (1, 0.5), (0, 1), (1, 2), (1, 1.5), (2, 0.5), (1, 1), (0, 1), (1, 0.5), (0, 0.5), (-2, 0.5), (-3, 0.5), (-2, 2), (-5, 1), (-5, 0.5), (-3, 0.5), (-3, 1), (-3, 1), (-2, 1), (-2, 0.5), (1, 0.5), (0, 2), (-2, 1), (-2, 0.5), (0, 0.5), (1, 0.5), (2, 0.5), (0, 0.5), (1, 0.5), (2, 2), (2, 2)]


In [7]:
# Loop the several list together until any one list don't have any more
first = ["a", "b", "c", "x"]
second = ["d", "e", "f", "y"]
third = ["g", "h", "i", "z"]
for one, two, three in zip(first, second, third):
    print((one, two, three))

('a', 'd', 'g')
('b', 'e', 'h')
('c', 'f', 'i')
('x', 'y', 'z')


In [8]:
len(int_music)

35

In [9]:
import numpy as np
arr_music = np.array(int_music)

In [10]:
arr_music[:, 0]

array([-2.,  2.,  2.,  2.,  2.,  1.,  0.,  1.,  1.,  2.,  1.,  0.,  1.,
        0., -2., -3., -2., -5., -5., -3., -3., -3., -2., -2.,  1.,  0.,
       -2., -2.,  0.,  1.,  2.,  0.,  1.,  2.,  2.])

In [11]:
arr_music.shape

(35, 2)

### The item will be set as object, if the lenth of item is different

In [12]:
dataset = np.array([ arr_music[:, 0][i:i+5] for i in range(arr_music.shape[0]-4) ])

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, y_train = dataset[:, 0:4], dataset[:, 4]

print(X_train.shape)
print(y_train.shape)

(31, 4)
(31,)


In [14]:
composer = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

composer.score(X_train, y_train)

0.967741935483871

In [15]:
init_music_notes = ['A-3', 'E-4', 'E-4', 'E-4']

new_music_int = [ map_note_to_int[x] for x in init_music_notes ]
for idx in range(35-4):
    new_music_int.append(int(composer.predict([new_music_int[-4:]])))
    
new_music_notes = [ (map_int_to_note[x], r ) for x, (_, r) in zip(new_music_int, int_music) ]
new_music_notes

[('A-3', 1),
 ('E-4', 1),
 ('E-4', 1),
 ('E-4', 1),
 ('E-4', 0.5),
 ('D-4', 0.5),
 ('C-4', 1),
 ('D-4', 2),
 ('C-4', 1.5),
 ('A-3', 0.5),
 ('G-3', 1),
 ('A-3', 1),
 ('E-3', 0.5),
 ('E-3', 0.5),
 ('G-3', 0.5),
 ('G-3', 0.5),
 ('G-3', 2),
 ('A-3', 1),
 ('A-3', 0.5),
 ('D-4', 0.5),
 ('C-4', 1),
 ('A-3', 1),
 ('A-3', 1),
 ('C-4', 0.5),
 ('D-4', 0.5),
 ('E-4', 2),
 ('C-4', 1),
 ('D-4', 0.5),
 ('E-4', 0.5),
 ('E-4', 0.5),
 ('D-4', 0.5),
 ('E-4', 0.5),
 ('D-4', 0.5),
 ('C-4', 2),
 ('D-4', 2)]

In [16]:
import time

rhythm = 2/4

# music = [('A-3',1), ('E-4',1), ('E-4',1), ('E-4',1), ('E-4',0.5), ('D-4',0.5), ('C-4',1), ('D-4',2),
#         ('D-4',1.5), ('E-4',0.5), ('D-4',1), ('C-4',1), ('D-4',0.5), ('C-4',0.5), ('A-3',0.5), ('G-3',0.5), ('A-3',2),
#         ('E-3',1), ('E-3', 0.5), ('G-3',.5), ('G-3',1), ('G-3',1), ('A-3',1), ('A-3',.5), ('D-4',.5), ('C-4', 2),
#         ('A-3',1), ('A-3', 0.5), ('C-4',.5), ('D-4',.5), ('E-4',.5), ('C-4',.5), ('D-4',.5), ('E-4',2), ('E-4',2)]

for i, r in new_music_notes:
    n = Note(i)
    n.channel = 1
    n.velocity = 100
    fluidsynth.play_Note(n)
    time.sleep(r*rhythm)